## TRPO
* trust region
* 이론 배경 많이 필요
* 이 알고리즘은 PG 와는 Policy Optimization 의 뿌리인데 전혀 다른 줄기로 새로운 알고리즘임
    * 안정적으로 파라미터를 업데이트하는데에 집중하는 알고리즘들 중 하나
    * NPG 처럼 안정적인 파라미터 업데이트 측면에 대한 연구
* kakade 의 논문 - 출발점
    * DP 에서 항상 성능이 보장되는 수식을 증명함
    * 이 식은 old policy 로 현재 traj 를 평가할 수 있음도 포함됨
    * 이 식은 time step 의 차원에서 정의된건데 관점을 바꿔 state 입장으로 바꿀 수 있음
    * 그럴 때 state 에 대한 policy 를 old 로 바꿔야 계산에 수월해 짐
    * 그게 가능한 이유는 old policy 근처에서는 둘 다 비슷하기 때문
* conservative policy iteration
    * 얼마나 비슷해야하는지는 잘 모름
    * old, now policy 를 섞어쓰는 policy 를 제시함
    * penalty 형태의 업데이트 식을 만듬
    * 이 penalty 식을 따라 업데이트를 하면 항상 성능 향상이 보장되도록 업데이트 됨
* general improvement
    * mixed policy 가 아닌 일반적인 policy 에 적용해야함
    * total variation divergence 를 이용
    * old, new policy 의 거리를 계산하여 이용 - lower bound
    * KLD 로 한 번 더 바꿈 - KL 이 TV^2 보다 항상 큼
    * 제약조건이 두 policy 의 차이가 특정 값보다 작도록 함.
    * 즉, 그냥 조금만 업데이트하도록 제한함
* practical 한 알고리즘
    * surrogate 함수를 최적화하는게 목적 함수를 최적화하는 것과 같음
    * 저 대리 함수가 계속 다루던 페널티 함수
    * penalty 함수를 보면 상수 C 가 너무 큰 값이라 업데이트가 실질적으로 불가능한 형태
    * penalty 식을 제약조건 식으로 풀어 헤침
    * 근데 이건 식 형태 자체를 바꾸는 거라서 엄밀성을 포기해버리는것임
    * 제약조건만 만족하면 목적함수를 최대한으로 업데이트 할 것임
    * 시그마로 표현된 식을 기댓값 형태로 바꿔서 샘플링이 가능하도록 해야함
    * I.S 등이 사용됨
* 최적화
    * 이렇게 제약조건 최적화 문제로 만들면 이제 최적화 문제를 푸는 line search 등을 쓰면 됨
    * conjugate gradient 를 위한 FIM 행렬 등도 쓰임

In [ ]:
import numpy as np

import torch
from torch.autograd import Variable
from utils import *


def conjugate_gradients(Avp, b, nsteps, residual_tol=1e-10):
    x = torch.zeros(b.size())
    r = b.clone()
    p = b.clone()
    rdotr = torch.dot(r, r)
    for i in range(nsteps):
        _Avp = Avp(p)
        alpha = rdotr / torch.dot(p, _Avp)
        x += alpha * p
        r -= alpha * _Avp
        new_rdotr = torch.dot(r, r)
        betta = new_rdotr / rdotr
        p = r + betta * p
        rdotr = new_rdotr
        if rdotr < residual_tol:
            break
    return x


def linesearch(model,
               f,
               x,
               fullstep,
               expected_improve_rate,
               max_backtracks=10,
               accept_ratio=.1):
    fval = f(True).data
    print("fval before", fval.item())
    for (_n_backtracks, stepfrac) in enumerate(.5**np.arange(max_backtracks)):
        xnew = x + stepfrac * fullstep
        set_flat_params_to(model, xnew)
        newfval = f(True).data
        actual_improve = fval - newfval
        expected_improve = expected_improve_rate * stepfrac
        ratio = actual_improve / expected_improve
        print("a/e/r", actual_improve.item(), expected_improve.item(), ratio.item())

        if ratio.item() > accept_ratio and actual_improve.item() > 0:
            print("fval after", newfval.item())
            return True, xnew
    return False, x


def trpo_step(model, get_loss, get_kl, max_kl, damping):
    loss = get_loss()
    grads = torch.autograd.grad(loss, model.parameters())
    loss_grad = torch.cat([grad.view(-1) for grad in grads]).data

    def Fvp(v):
        kl = get_kl()
        kl = kl.mean()

        grads = torch.autograd.grad(kl, model.parameters(), create_graph=True)
        flat_grad_kl = torch.cat([grad.view(-1) for grad in grads])

        kl_v = (flat_grad_kl * Variable(v)).sum()
        grads = torch.autograd.grad(kl_v, model.parameters())
        flat_grad_grad_kl = torch.cat([grad.contiguous().view(-1) for grad in grads]).data

        return flat_grad_grad_kl + v * damping

    stepdir = conjugate_gradients(Fvp, -loss_grad, 10)

    shs = 0.5 * (stepdir * Fvp(stepdir)).sum(0, keepdim=True)

    lm = torch.sqrt(shs / max_kl)
    fullstep = stepdir / lm[0]

    neggdotstepdir = (-loss_grad * stepdir).sum(0, keepdim=True)
    print(("lagrange multiplier:", lm[0], "grad_norm:", loss_grad.norm()))

    prev_params = get_flat_params_from(model)
    success, new_params = linesearch(model, get_loss, prev_params, fullstep,
                                     neggdotstepdir / lm[0])
    set_flat_params_to(model, new_params)

    return loss